# 自動機械学習で分類モデルを訓練する

モデルを訓練するために使用できる機械学習アルゴリズムには多くの種類があり、特定のデータと予測要件に最も効果的なアルゴリズムを決定するのは簡単ではない場合があります。さらに、正規化、欠損特徴インピュテーションなどのテクニックを使って学習データを前処理することで、モデルの予測性能に大きな影響を与えることができます。要件に最適なモデルを見つけようとすると、アルゴリズムと前処理変換の多くの組み合わせを試す必要があるかもしれません。

Azure Machine Learningでは、異なるアルゴリズムと前処理オプションを使用してトレーニングされたモデルの比較を自動化できます。この機能を活用するには、[Azure Machine Learning Studio](https://ml/azure.com)またはPython SDK(v2)のビジュアルインターフェイスを使用できます。Python SDKでは、自動機械学習ジョブの設定をより詳細に制御できますが、ビジュアルインターフェースの方が使いやすくなっています。

## Before you start

You'll need the latest version of the  **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## ワークスペースに接続する

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [7]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [8]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## データの準備

自動機械学習用の学習スクリプトを作成する必要はないが、学習データをロードする必要がある。

今回は、糖尿病患者の詳細を含むデータセットを使用します。

自動機械学習の入力としてデータセットを渡すには、そのデータは表形式で、対象となる列が含まれていなければなりません。データを表形式のデータセットとして解釈するためには、入力データセットは **MLTable** でなければなりません。

MLTableデータ資産は、セットアップ時に既に作成されています。このデータ資産は、**Data** ページに移動して調べることができます。ここで名前 `diabetes-training-table` とバージョン `1` を指定してデータアセットを取得します。

In [3]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## 自動機械学習ジョブの設定

これで、自動機械学習実験を設定する準備ができた。

以下のコードを実行すると、以下のような自動機械学習ジョブが作成される：

- aml-cluster`という名前の計算クラスタを使う。
- ターゲットカラムとして `Diabetic` を設定する。
- 主要な指標として `accuracy` を設定する。
- 総トレーニング時間 `60` 分後にタイムアウトする。
- 最大5`のモデルを学習する。
- LogisticRegression` アルゴリズムではモデルは学習されない。

In [4]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [9]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/gray_nutmeg_s9240p5fg4?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


While the job is running, you can monitor it in the Studio.